# 7 Predicción con un modelo SARIMA que supone una tendencia constante en el tiempo, Arima(0,0,1)(0,1,1)

In [1]:
%%HTML
<b>Sección anterior: </b>
<a href="http://nbviewer.jupyter.org/github/jaircastruita/visualizations/blob/master/SARIMA_011_011_forecast.ipynb">
Un modelo de predicción con una tendencia que varía con el tiempo, SARIMA(0,1,1)(0,1,1).</a>

En el notebook anterior la serie de tiempo fue analizada utilizando un modelo SARIMA que supone una tendencia variable en tiempo. Esta vez la aproximación será utilizando un modelo que asuma una tendencia constante. Como en el notebook pasado se tratará la información de la misma manera, solamente se diferirá en el modelo utilizado.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns; sns.set()
import datetime as dt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18, 10

In [2]:
import statsmodels.api as sm
import rpy2.robjects as R
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [3]:
k_3 = pd.read_csv(r'cluster_member3.csv')
res = pd.read_csv(r'phase1_no_normalized.csv').T

In [4]:
res['cluster'] = k_3.values
temp_res = res.sort_values(['cluster'])

In [5]:
path = "ecobici.csv"
trips = pd.read_csv(path,
                    parse_dates=['date_removed', 'date_arrived'],
                    infer_datetime_format=True,
                    low_memory=False)

# Find the start date
ind = pd.DatetimeIndex(trips.date_arrived)
trips['date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour

trips = trips.loc[trips['action'] != 'C  ']

idx = pd.date_range(trips.date.min(), trips.date.max() + dt.timedelta(days=1), freq='H')

by_hour = (trips
     .set_index('date_arrived')
     .groupby([pd.TimeGrouper('H'), 'station_arrived'])
     .station_arrived
     .count()
     .unstack()
     .reindex(idx)
     .fillna(0)
     )
# Delete activity of non existent station's IDs in the dataset
by_hour.drop([col for col in list(by_hour.columns.values) if col > 275], axis=1, inplace=True)

# Keep only weekdays to make a more stable count signal
by_hour = by_hour[by_hour.index.dayofweek < 5]

# Since the system's opening, the slow activity of the first days were removed
by_hour = by_hour['2010-03-01':]

# Keep business hours for the system. Ecobici closes from 0:00 to 5:00
by_hour = by_hour.between_time('6:00','23:00')

In [6]:
logged_ts = by_hour + 1
logged_ts = logged_ts.apply(np.log)

# deal with missing values. see issue
logged_ts.interpolate(inplace=True)

In [7]:
#80-20 Train & test
start = (len(logged_ts) * 80) / 100

train = logged_ts.iloc[:start]
test = logged_ts.iloc[start:]

In [8]:
head = temp_res[temp_res.cluster == 1]
body = temp_res[temp_res.cluster == 2].sample(10)
tail = temp_res[temp_res.cluster == 3]

sub_sample = pd.concat([head,body,tail])

In [9]:
%load_ext rpy2.ipython

In [10]:
stats = importr('stats')
tseries = importr('tseries')
forecast = importr('forecast')

# Ajuste y predicción del modelo

Una vez más se realizarán las predicciones, solo que ahora con el modelo $Arima(0,0,1)(0,1,1)_{18}$. El número de coeficientes que se decidió utilizar se determinó por el comportamiento de los correlogramas ACF y PACF. También en el notebook pasado se explicó el por qué se utilizará el modelo SARIMA sin diferenciar el su parte no Estacional:

$$\hat{Y} = \mu + Y_{t-18} + \phi _1 (Y_{t-1} - Y_{t-19})$$

donde $\phi_1$ denota el coeficiente AR(1) y $\mu$ la constante de la media en la actividad estacional durante el tiempo. Al no depender de las observaciones anteriores este método es más estable a la hora de realizar predicciones a largo plazo.

In [11]:
from sklearn.metrics import mean_absolute_error

In [12]:
order = R.IntVector((0,0,1))
season = R.ListVector({'order': R.IntVector((0,1,1)), 'period' : 18})

columns = ['AIC','Durbin-Watson','RSS train','Unit root','Mean Absolute Error']
df_ = pd.DataFrame(index = sub_sample.index.astype(int),columns=columns)
df_ = df_.fillna(0) # with 0s rather than NaNs

forecast_matrix = pd.DataFrame(index = test.index, columns=sub_sample.index.astype(int))
forecast_matrix = forecast_matrix.fillna(0)
    
arima_models = []

for i in sub_sample.index.astype(int):
    r_df = pandas2ri.py2ri(train[[i]])
    y = stats.ts(r_df)
    model = stats.arima(y, order = order, seasonal=season)
    
    %Rpush model
    AIC = R.r('model$aic')[0]
    durbin_watson = sm.stats.durbin_watson(R.r('model$residuals'))
    RSS = sum((train[[i]].values-R.r('fitted(model)'))**2)
    unit_root = sum(R.r('model$coef'))
    
    f = forecast.forecast(model,len(test))
    pred = [j[1] for j in f[3].items()]
    dt = test.index
    pr = pd.Series(pred, index = dt)
    mae = mean_absolute_error(test[i],pr)
    forecast_matrix[i] = pred

    df_.loc[i] = [AIC, durbin_watson, RSS, unit_root, mae]
    arima_models.append(model)

exited successful


In [13]:
df_

,AIC,Durbin-Watson,RSS train,Unit root,Mean Absolute Error
1,19632.286863,1.922442,3286.520807,-0.686012,0.413369
64,18785.491516,1.914788,3099.480342,-0.656720,0.401445
27,22393.865558,1.866739,3984.067525,-0.529957,0.537152
36,18447.217239,1.913822,3027.874546,-0.667720,0.525074
13,23610.433619,1.888506,4330.513634,-0.633651,0.485926
6,21753.883537,1.967301,3807.360319,-0.780715,0.483698
16,23707.689107,1.927024,4359.063533,-0.723556,0.561621
73,21489.014917,1.960047,3737.766220,-0.754954,0.419417
70,23692.277715,1.970685,4354.678690,-0.779084,0.440119
25,22512.999222,1.946245,4012.612996,-0.746459,0.549742


Como en el notebook pasado, los resultados obtenidos muestran que los modelos no contienen una raiz unitaria. Aunque el error obtenido por el RSS en el conjunto de entrenamiento es menor para los elementos que conforman el head y tail los valores MAE entre éstos no presentan una diferencia distinguible.

In [14]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

### Predicciones en head

In [15]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[1], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[1], color='red', alpha=0.5)

show(p)

### Predicciones en body

In [16]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[88], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[88], color='red', alpha=0.5)

show(p)

### Predicciones en tail

In [17]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[64], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[64], color='red', alpha=0.5)

show(p)

Puede apreciarse que en comparación con las gráficas pasadas, éstas no presentan una tendencia que dirija a la predicción a un aumento o decremento gradual, sin embargo la vuelve menos sensible ante los cambios en las diferentes horas del día.

# Conclusiones hasta el momento

- El modelo SARIMA que considera una tendencia no constante es mejor que el modelo arima que no lo considera pero solo para pronósticos a corto plazo. Una vez utilizado el mismo modelo para largo plazo la predicción se degrada rápidamente.
- Aunque utilizando MAE los valores obtenidos no han sido demasiado significativos en ninguno de los casos se logra percibir que los modelos ajustados para las estaciones pertenecientes a la cabeza logran modelar mejor las oscilaciones de uso durante el día.

In [2]:
from IPython.display import HTML
#To hide code from the nbviewer render
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')